# Dataset
Για τη δημιουργία του dataset επιλέχθηκαν 2 κλάσεις του places, ***kitchen*** και ***office***. Οι εικόνες του COCO dataset που αντιστοιχούν σε αυτές τις κλάσεις επιλέχθηκαν με βάση κατηγορίες αντικειμένων που περιέχουν. Συγκεκριμένα για την κλάση ***kitchen*** επιλέχθηκαν οι εικόνες που έχουν ως labels τις κατηγορίες ***refrigerator,sink,toaster,oven,microwave*** και αντίστοιχα για την κλάση ***office*** οι εικόνες με labels ***keyboard, laptop, mouse, cell phone, tv, book***. Για την κλάση *kitchen* συγκεντρώθηκαν 31 εικόνες ενώ για την κλάση *office* 62. 

In [ ]:
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip # download dataset
!unzip annotations_trainval2017.zip # unzip dataset
from pycocotools.coco import COCO
coco = COCO('annotations/instances_train2017.json')

--2022-07-24 19:47:07--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.45.36
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.45.36|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip’

annotations_trainva 100%[===================>] 241.19M  98.3MB/s    in 2.5s    

2022-07-24 19:47:10 (98.3 MB/s) - ‘annotations_trainval2017.zip’ saved [252907541/252907541]

Archive:  annotations_trainval2017.zip
  inflating: annotations/instances_train2017.json  
  inflating: annotations/instances_val2017.json  
  inflating: annotations/captions_train2017.json  
  inflating: annotations/captions_val2017.json  
  inflating: annotations/person_keypoints_train2017.json  
  inflating: annotations/person_keypoints_val2017.json  
loading annotations into memory...
Done (t=22.41s)
creating index...
ind

In [ ]:
 %matplotlib inline
import matplotlib.pyplot as plt
import pylab
import os
pylab.rcParams['figure.figsize'] = (15.0, 17.0)
import torch
from torch.autograd import Variable as V
import torchvision.models as models
from torchvision import transforms as trn
from torch.nn import functional as F
import os
import numpy as np
import cv2
from PIL import Image
from skimage import io
from IPython import display


In [ ]:
# get all images containing given categories

classes = {
    'kitchen' : ['refrigerator','sink','toaster','oven','microwave'],
    'office' : ['keyboard', 'laptop', 'mouse', 'cell phone','tv' , 'book']
}
catIds ={'kitchen': [], 'office': []}
imgIds ={'kitchen': [], 'office': []}
img = {'kitchen': [], 'office': []}
for cat in ['kitchen','office']:
  catIds[cat] = coco.getCatIds(catNms=classes[cat])
  imgIds[cat] = coco.getImgIds(catIds = catIds[cat] )
  img [cat]= coco.loadImgs(imgIds[cat]) #list [{coco_url,..,id,...},..]
  print('The "' + cat + '" class has %d images' %len(img[cat]))

The "kitchen" class has 31 images
The "office" class has 62 images


In [ ]:
# Get all anotations of an image 

anns_per_image = {'kitchen': {}, 'office':{}} # imgId --> label_ids=[], label_texts=[]
for cat in ['kitchen','office']:

  for imgId in imgIds[cat]:
      ann_ids = set(coco.getAnnIds(imgIds = imgId, iscrowd=None))
      anns = coco.loadAnns(ann_ids)
      anns_per_image[cat][imgId]={"label_ids":ann_ids, "label_texts":[coco.loadCats(ann['category_id'])[0]['name'] for ann in anns] }



# Classification / XAI
Στο dataset των εικόνων που δημιουργήθηκε εφαρμόζουμε τον classifier του places, ο οποίος κατατάσσει τις εικόνες βάσει πιθανοτήτων σε κάποιες από τις κατηγορίες του. Για κάθε εικόνα παρουσιάζονται τα labels/annotations τα οποία διαθέτει από το COCO dataset, οι 5 πιο πιθανές κατηγορίες (scene categories) στις οποίες κατατάχθηκε από τον classifier καθώς και τα scene attributes της από το places. Εφαρμόζοντας το GradCam σημειώνονται οι περιοχές της εικόνας με τα χαρακτηριστικά που οδήγησαν στην πρόβλεψη που έκανε ο classifier. 

In [ ]:
# PlacesCNN to predict the scene category, attribute, and class activation map in a single pass
# by Bolei Zhou, sep 2, 2017

def recursion_change_bn(module):
    if isinstance(module, torch.nn.BatchNorm2d):
        module.track_running_stats = 1
    else:
        for i, (name, module1) in enumerate(module._modules.items()):
            module1 = recursion_change_bn(module1)
    return module

def load_labels():
    # prepare all the labels
    # scene category relevant
    file_name_category = 'categories_places365.txt'
    if not os.access(file_name_category, os.W_OK):
        synset_url = 'https://raw.githubusercontent.com/csailvision/places365/master/categories_places365.txt'
        os.system('wget ' + synset_url)
    classes = list()
    with open(file_name_category) as class_file:
        for line in class_file:
            classes.append(line.strip().split(' ')[0][3:])
    classes = tuple(classes)

  

    # scene attribute relevant
    file_name_attribute = 'labels_sunattribute.txt'
    if not os.access(file_name_attribute, os.W_OK):
        synset_url = 'https://raw.githubusercontent.com/csailvision/places365/master/labels_sunattribute.txt'
        os.system('wget ' + synset_url)
    with open(file_name_attribute) as f:
        lines = f.readlines()
        labels_attribute = [item.rstrip() for item in lines]
    file_name_W = 'W_sceneattribute_wideresnet18.npy'
    if not os.access(file_name_W, os.W_OK):
        synset_url = 'http://places2.csail.mit.edu/models_places365/W_sceneattribute_wideresnet18.npy'
        os.system('wget ' + synset_url)
    W_attribute = np.load(file_name_W)

    return classes, labels_attribute, W_attribute

def hook_feature(module, input, output):
    features_blobs.append(np.squeeze(output.data.cpu().numpy()))

def returnCAM(feature_conv, weight_softmax, class_idx):
    # generate the class activation maps upsample to 256x256
    size_upsample = (256, 256)
    nc, h, w = feature_conv.shape
    output_cam = []
    for idx in class_idx:
        cam = weight_softmax[class_idx].dot(feature_conv.reshape((nc, h*w)))
        cam = cam.reshape(h, w)
        cam = cam - np.min(cam)
        cam_img = cam / np.max(cam)
        cam_img = np.uint8(255 * cam_img)
        output_cam.append(cv2.resize(cam_img, size_upsample))
    return output_cam

def returnTF():
# load the image transformer
    tf = trn.Compose([
        trn.Resize((224,224)),
        trn.ToTensor(),
        trn.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    return tf


def load_model():
    # this model has a last conv feature map as 14x14

    model_file = 'wideresnet18_places365.pth.tar'
    if not os.access(model_file, os.W_OK):
        os.system('wget http://places2.csail.mit.edu/models_places365/' + model_file)
        os.system('wget https://raw.githubusercontent.com/csailvision/places365/master/wideresnet.py')

    import wideresnet
    model = wideresnet.resnet18(num_classes=365)
    checkpoint = torch.load(model_file, map_location=lambda storage, loc: storage)
    state_dict = {str.replace(k,'module.',''): v for k,v in checkpoint['state_dict'].items()}
    model.load_state_dict(state_dict)
    
    for i, (name, module) in enumerate(model._modules.items()):
        module = recursion_change_bn(model)
    model.avgpool = torch.nn.AvgPool2d(kernel_size=14, stride=1, padding=0)
    
    model.eval()

    # hook the feature extractor
    features_names = ['layer4','avgpool'] # this is the last conv layer of the resnet
    for name in features_names:
        model._modules.get(name).register_forward_hook(hook_feature)
    return model


# load the labels
classes, labels_attribute, W_attribute = load_labels()

# load the model
features_blobs = []
model = load_model()

# load the transformer
tf = returnTF() # image transformer

# get the softmax weight
params = list(model.parameters())
weight_softmax = params[-2].data.numpy()
weight_softmax[weight_softmax<0] = 0


In [ ]:
import os
os.makedirs("heatmaps")
os.makedirs("heatmaps/office")
os.makedirs("heatmaps/kitchen")

def read_image_from_url(img_url):
    img = io.imread(img_url)  #numpy.ndarray
    img = Image.fromarray(img) #PIL Image   image_from_array.show()
    return img

In [ ]:
def classify (img_url):
  image =read_image_from_url(img_url)
  input_img = V(tf(image).unsqueeze(0))

  # forward pass
  logit = model.forward(input_img)
  h_x = F.softmax(logit, 1).data.squeeze()
  probs, idx = h_x.sort(0, True)
  probs = probs.numpy()
  idx = idx.numpy()
  # output the prediction of scene category
  print('--SCENE CATEGORIES:')
  for i in range(0, 5):
      print('{:.3f} -> {}'.format(probs[i], classes[idx[i]]))

  # output the scene attributes
  responses_attribute = W_attribute.dot(features_blobs[1])
  idx_a = np.argsort(responses_attribute)
  print('--SCENE ATTRIBUTES:')
  print(', '.join([labels_attribute[idx_a[i]] for i in range(-1,-10,-1)]))
  # generate class activation mapping

  #print('Class activation map is saved in "heatmaps/'+cat+'/heatmap_' + str(j) +'.jpg"')
  CAMs = returnCAM(features_blobs[0], weight_softmax, [idx[0]])

  # render the CAM and output
  #img = cv2.imread('test.jpg')
  imgs = io.imread(img_url)
  height, width, _ = imgs.shape
  heatmap = cv2.applyColorMap(cv2.resize(CAMs[0],(width, height)), cv2.COLORMAP_JET)
  result = heatmap * 0.4 + imgs * 0.5

  imagepath = './heatmaps/'+cat+ '/heatmap_' + str(j) +'.jpg'
  cv2.imwrite(imagepath, result)
  return imagepath

Όπως φαίνεται και από τα heatmaps που πορκείπτουν από το gradcam 

In [3]:
import matplotlib.image as mpimg
for cat in ['kitchen','office']:
  print(cat)
  for j in range(25):
    img_url = img[cat][j]['coco_url']
    print("Image %s_%d" %(cat,j) )
    print('Annotations', anns_per_image[cat][img[cat][j]['id']]['label_texts'])
    heatpath = classify(img_url)
    print(heatpath)
    print('RESULT ON ' + img_url)
    image = read_image_from_url( img_url )
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.axis('off')
    # Image.open(imagepath).show()
    #display.Image(imagepath)
    img_heatmap = mpimg.imread(heatpath)
    plt.subplot(1, 2, 2) # index 2
    plt.imshow(img_heatmap)
    plt.axis('off')
    plt.show()

In [4]:
for cat in ['kitchen']:#,'office']:
  print(cat)
  for j in range(25,31):
    img_url = img[cat][j]['coco_url']
    print("Image %s_%d" %(cat,j) )
    print('Annotations', anns_per_image[cat][img[cat][j]['id']]['label_texts'])
    heatpath = classify(img_url)
    print(heatpath)
    print('RESULT ON ' + img_url)
    image = read_image_from_url( img_url )
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.axis('off')
    # Image.open(imagepath).show()
    #display.Image(imagepath)
    img_heatmap = mpimg.imread(heatpath)
    plt.subplot(1, 2, 2) # index 2
    plt.imshow(img_heatmap)
    plt.axis('off')
    plt.show()

In [5]:
def image_classification(imgCat, imageId):
  for ii in img[imgCat]:
    if ii['id'] == imageId :
      img_url = ii['coco_url']
      print(img_url)
      print('Annotations', anns_per_image[imgCat][ii['id']]['label_texts'])
      heatpath = classify(img_url)
      print("printed heatpath")
      print(heatpath)
      print('RESULT ON ' + img_url)
      image = read_image_from_url( img_url )
      plt.subplot(1, 2, 1)
      plt.imshow(image)
      plt.axis('off')
      img_heatmap = mpimg.imread(heatpath)
      plt.subplot(1, 2, 2) # index 2
      plt.imshow(img_heatmap)
      plt.axis('off')
      plt.show()

In [6]:
for cat in ['office']:#,'office']:
  print(cat)
  for j in range(61):
    img_url = img[cat][j]['coco_url']
    print("Image %s_%d" %(cat,j) )
    print('Annotations', anns_per_image[cat][img[cat][j]['id']]['label_texts'])
    heatpath = classify(img_url)
    print(heatpath)
    print('RESULT ON ' + img_url)
    image = read_image_from_url( img_url )
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.axis('off')
    # Image.open(imagepath).show()
    #display.Image(imagepath)
    img_heatmap = mpimg.imread(heatpath)
    plt.subplot(1, 2, 2) # index 2
    plt.imshow(img_heatmap)
    plt.axis('off')
    plt.show()

# Γνώση και wordnet

In [ ]:
!pip install owlready2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 25.4 MB 30.6 MB/s 
  Created wheel for owlready2: filename=Owlready2-0.38-cp37-cp37m-linux_x86_64.whl size=22127111 sha256=8965af865787de8cce018a0b0d5a67f7b6ed46751709b2ee7adf5cb0363d592a
  Stored in directory: /root/.cache/pip/wheels/ee/48/58/3ea20d69efcb97a26048fbfcce9332412077df80d4ae52365b
Successfully built owlready2


In [ ]:
from owlready2 import *
from nltk.corpus import wordnet as wn
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
# print(list(onto.properties()))
# print(list(onto.classes()))
# print(list(onto.individuals()))

[]
[]
[]


To dataset ενώνεται με το wordnet βρίσκοντας συνώνυμα και υπερκλάσεις του κάθε label που χαρακτηρίζει μια εικόνα. Για την αναπαράσταση αυτής της γνώσης σε .οwl δημιουργούμε μια οντολογία για την κάθε κατηγορία (kitchen και office). Στην κάθε οντολογία υπάρχουν οι κλάσεις *Image* και *DepictedObejct* ως υποκλάσεις της κλάσης *Thing* της οντολογίας. Ορίζεται επίσης ο ρόλος *hasObject* μεταξύ των προαναφερθέντων κλάσεων. Για κάθε εικόνα δημιουργείται ένα στιγμιότυπο (indivindual/instance) τύπου *Image* με αναγνωριστικό το id της εικόνας. Για κάθε αντικείμενο της εικόνας δημιουργούμε στιγμιότυπο τύπου *DepictedObject* και συνδέουμε την εικόνα με αυτό μέσω του ρόλου *hasObject*. Για το πρώτο synset(συνώνυμο) από το wordnet του εκάστοτε αντικειμένου δημιουργούμε μια κλάση όπως και για όλα τα υπερώνυμά του, διατηρώντας την ιεραρχία μεταξύ τους. Το αντικείμενο ορίζεται ως τύπου της κλάσης του synset και κατά συνέπεια ως τύπου όλων των υπερκλάσεων αυτού. 

In [ ]:
for i in onto.individuals():
    destroy_entity(i)
for i in onto.classes():
    destroy_entity(i)
for i in onto.properties():
    destroy_entity(i)    

In [2]:
for cat in  ['office','kitchen' ]:
  #create an ontology with the IRI (identifier) "http://myontology/kitchen" and "http://myontology/office"
  ontology = 'http://myontology/'+cat
  onto = get_ontology('http://myontology/'+cat)
  with onto:
    ## Ορισμός εννοιών για εικόνες και αντικείμενα
    class Image(Thing): # Image subclass of owl's class Thing
      namespace=onto
      pass

    class DepictedObject(Thing): # DepictedObject subclass of owl's class Thing
      namespace=onto
      pass

  ## Ορισμός ρόλου "hasObject" (ποιες εικόνες περιέχουν ποια αντικείμενα)
    class hasObject(Image>>DepictedObject):
      namespace=onto
      pass


  ## ids των 100 πρώτων εικόνων
  ids = imgIds[cat][:100]
  ## Για κάθε εικόνα
  for im_id in ids:
      ## Δημιουργία individual τύπου "Image"
      #im = onto['Image'](str(im_id))
      im = Image(str(im_id))

      ## Για κάθε αντικείμενο στην εικόνα:
      for obj_name in anns_per_image[cat][im_id]['label_texts']:
          if " " in obj_name:
              obj_name = obj_name.replace(" ", "_")
              
          ## Εύρεση πρώτου synset στο wordnet
          synsets = wn.synsets(obj_name)
          if len(synsets)<1:
              continue
          
          ## Δημιουργία individual τύπου "Object"
          obj = DepictedObject()

          ## Σύνδεση της εικόνας με το αντικείμενο μέσω του ρόλου hasObject
          im.hasObject.append(obj)

          synset = synsets[0].name()
          if (obj_name == "mouse"):
            synset = 'mouse.n.04'
          elif (obj_name == "microwave"):
            synset = synsets[1].name()
          elif (obj_name == "toaster"):
            synset = synsets[1].name()
          ## Αν υπάρχει η αντίστοιχη έννοια στη γνώση
          if onto[synset] in onto.classes():

              ## Ορίζουμε το αντικείμενο obj ως τύπου "synset"
              obj.is_a.append(onto[synset])
              ## Αν δεν υπάρχει η αντίστοιχη έννοια στη γνώση, την ορίζουμε, μαζί με τις υπερέννοιές της
          else:
              print("hypenymssss")
              ## Εύρεση υπερώνυμων
              hyper = lambda s:s.hypernyms()
              hypers = [s.name() for s in list(wn.synset(synset).closure(hyper))]
              hypers = reversed(hypers)
              ## Ορισμός ιεραρχίας εννοιών
              father = Thing
              for h in hypers:
                if onto[h] not in onto.classes():
                    with onto:
                        cl = types.new_class(h,(father,))
                    #father = onto[h]
                father = onto[h]
              if onto[synset] not in onto.classes():
                  with onto:
                      cl = types.new_class(synset,(father,))
                  if(obj_name == "microwave"):
                      print("First class for microwave with father" , father)
                  ## Ορίζουμε το αντικείμενο obj ως τύπου "synset"
              with onto:
                  if(obj_name == 'microwave'):
                    print("Microwave is_a", onto[synset])
                  obj.is_a.append(onto[synset])

      ## Αποθηκεύουμε την οντολογία
      onto.save('myonto_'+cat+'.nt',format='ntriples')
      